### Testing pose detection

In [2]:
import cv2
import mediapipe as mp

In [ ]:
# Initialize MediaPipe Pose solution.
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize MediaPipe drawing utility.
mp_drawing = mp.solutions.drawing_utils

# Capture video from the default camera.
cap = cv2.VideoCapture("/Users/varunshankarhoskere/Downloads/WhatsApp Video 2023-11-25 at 13.16.04.mp4")

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # Convert the BGR image to RGB.
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image and detect the pose.
    results = pose.process(image)

    #print(results.x, results.y, results.z)

    print(results)

    """
    results object contains the x y and z axes values of the point that will be drawn
                                visibility number which indicated whether that point is clear
    """


    # Draw the pose annotations on the image.
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Display the resulting image.
    cv2.imshow('MediaPipe Pose', image)

    # Break the loop if 'q' is pressed.
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

pose.close()
cap.release()


### Relevant code : 

In [3]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm

Function to get the pose from the frame

In [4]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, enable_segmentation=False, min_detection_confidence=0.5)

def extract_pose_data(frame):
    # Convert the color space from BGR to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame to detect the pose
    results = pose.process(frame_rgb)

    # Check if pose landmarks are detected
    if results.pose_landmarks:
        # Extract landmarks data
        pose_data = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark]).flatten()
        return pose_data
    return np.zeros(132)  # Assuming 33 landmarks each with 4 values (x, y, z, visibility)


I0000 00:00:1701174564.891697       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Generating the sequences from the dataset

In [ ]:
def create_image_sequences(image_folder, sequence_length):
    sequences = []
    image_files = sorted(os.listdir(image_folder))

    for i in tqdm(range(len(image_files) - sequence_length + 1), desc="Processing image sequences"):
        sequence = []

        for j in range(i, i + sequence_length):
            img_path = os.path.join(image_folder, image_files[j])
            frame = cv2.imread(img_path)
            if frame is None:
                print(f"Failed to read image: {img_path}")
                break
            pose_data = extract_pose_data(frame)
            sequence.append(pose_data)

        if len(sequence) == sequence_length:
            sequences.append(sequence)

    return np.array(sequences)

def create_label_sequences(label_folder, sequence_length, image_files):
    labels = []

    for i in tqdm(range(len(image_files) - sequence_length + 1), desc="Processing label sequences"):
        sequence_labels = []

        for j in range(i, i + sequence_length):
            label_path = os.path.join(label_folder, image_files[j].replace('.png', '.txt'))
            with open(label_path, 'r') as file:
                if file is None:
                    print(f"failed to read label: {label_path}")
                fall_label = int(file.read().split()[0])
                sequence_labels.append(fall_label)

        # Use the most frequent label in the sequence as the sequence label
        if sequence_labels:
            labels.append(max(set(sequence_labels), key=sequence_labels.count))

    return np.array(labels)

# Example usage
sequence_length = 30
image_folder = "/Users/varunshankarhoskere/Desktop/Academics/PES/Capstone/Datasets/images"
label_folder = "/Users/varunshankarhoskere/Desktop/Academics/PES/Capstone/Datasets/labels"

image_sequences = create_image_sequences(image_folder, sequence_length)
image_files = sorted(os.listdir(image_folder))  # Ensure this matches the sorting in image sequence creation
label_sequences = create_label_sequences(label_folder, sequence_length, image_files)

# Now image_sequences and label_sequences contain your data


In [ ]:
import numpy as np

# Saving the image sequences
np.save('/path/to/save/image_sequences.npy', image_sequences)

# Saving the label sequences
np.save('/path/to/save/label_sequences.npy', label_sequences)

In [ ]:
# Loading the image sequences
image_sequences = np.load('image_sequences.npy')

# Loading the label sequences
label_sequences = np.load('label_sequences.npy')

In [7]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [9]:
# Convert sequences and labels to PyTorch tensors
sequences_tensor = torch.tensor(image_sequences, dtype=torch.float32)
labels_tensor = torch.tensor(label_sequences, dtype=torch.long)

# Create a dataset and data loader
dataset = TensorDataset(sequences_tensor, labels_tensor)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# LSTM model initialization and training loop
# (Use the FallDetectionLSTM class and training loop provided earlier)


In [10]:
import torch.nn as nn

class FallDetectionLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(FallDetectionLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Initialize hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


In [11]:
from sklearn.model_selection import train_test_split

# Assuming image_sequences and label_sequences are your data
train_seqs, test_seqs, train_labels, test_labels = train_test_split(
    image_sequences, label_sequences, test_size=0.2, random_state=42
)

train_dataset = TensorDataset(torch.tensor(train_seqs, dtype=torch.float32), torch.tensor(train_labels, dtype=torch.long))
test_dataset = TensorDataset(torch.tensor(test_seqs, dtype=torch.float32), torch.tensor(test_labels, dtype=torch.long))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [12]:
import torch.optim as optim

In [15]:
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        for sequences, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

def evaluate_model(model, test_loader):
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for sequences, labels in test_loader:
            outputs = model(sequences)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')

input_size = 132
hidden_size = 132
num_layers = 3
num_classes = 2

# Model Initialization
model = FallDetectionLSTM(input_size, hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training and Evaluation
train_model(model, train_loader, criterion, optimizer, num_epochs=10)
evaluate_model(model, test_loader)


Epoch [1/10], Loss: 0.2466
Epoch [2/10], Loss: 0.0670
Epoch [3/10], Loss: 0.0312
Epoch [4/10], Loss: 0.1829
Epoch [5/10], Loss: 0.0271
Epoch [6/10], Loss: 0.0004
Epoch [7/10], Loss: 0.0562
Epoch [8/10], Loss: 0.0281
Epoch [9/10], Loss: 0.0881
Epoch [10/10], Loss: 0.2218
Accuracy: 98.51%


In [20]:
torch.save(model.state_dict(), 'lstm_model.pth')

In [22]:
import cv2
import mediapipe as mp

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, enable_segmentation=False, min_detection_confidence=0.5)

def extract_pose_data(frame):
    # Process frame with MediaPipe Pose
    results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    # Extract pose landmarks or other relevant data
    # This should match the format of your training data
    # ...
    if results.pose_landmarks:
        # Extract pose landmarks and flatten the array
        pose_data = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark]).flatten()
        return pose_data
    return np.zeros(132)

cap = cv2.VideoCapture('/Users/varunshankarhoskere/Downloads/WhatsApp Video 2023-11-25 at 13.16.04.mp4')
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
sequence_length = 30  # Assuming this is the sequence length used during training

all_sequences = []
current_sequence = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    pose_data = extract_pose_data(frame)
    current_sequence.append(pose_data)

    if len(current_sequence) == sequence_length:
        all_sequences.append(current_sequence.copy())
        current_sequence.pop(0)  # Slide the window

cap.release()


I0000 00:00:1701169875.488914       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro


In [23]:
# Assuming your model class is FallDetectionLSTM
# and you have a saved model file 'lstm_model.pth'
model = FallDetectionLSTM(input_size, hidden_size, num_layers, num_classes)
model.load_state_dict(torch.load('lstm_model.pth'))
model.eval()

# Convert sequences to tensor
sequences_tensor = torch.tensor(all_sequences, dtype=torch.float32)

# Make predictions
with torch.no_grad():
    for sequence in sequences_tensor:
        sequence = sequence.unsqueeze(0)  # Add batch dimension
        output = model(sequence)
        _, predicted = torch.max(output.data, 1)
        print(predicted)
        # Now `predicted` is your model's prediction for this sequence
        # ...


/var/folders/6r/rq8wfq31581cybh61k1_sz8m0000gn/T/ipykernel_870/3515009451.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:264.)
  sequences_tensor = torch.tensor(all_sequences, dtype=torch.float32)


tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tens

In [25]:
model = FallDetectionLSTM(input_size, hidden_size, num_layers, num_classes)
model.load_state_dict(torch.load('lstm_model.pth'))
model.eval()

# Assuming sequences_tensor is your input data
with torch.no_grad():
    for sequence in sequences_tensor:
        sequence = sequence.unsqueeze(0)
        output = model(sequence)
        predicted_class = torch.sigmoid(output).round()  # For binary classification with sigmoid
        # or use torch.max for softmax
        print(predicted_class)


tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0., 0.]])
tensor([[0.,